In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models,Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,MaxPooling1D,Conv1D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import Flatten, Input, concatenate
import scipy as sp
# !pip install spafe==0.2.0
import spafe
from spafe.features.gfcc import gfcc
from scipy.stats import kurtosis,skew,mode,gstd,describe,iqr,gmean,hmean,median_abs_deviation,variation,tstd,gstd,moment,entropy
import librosa, librosa.display
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics as skm
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam 

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Eusipco Paper/mvd_gfcc.csv')
X_train=np.load('/content/drive/MyDrive/Eusipco Paper/mvd_gfcc.npy',allow_pickle=True)
Y_train=np.array(df['vehicle'].tolist())

In [ ]:
df['vehicle'].value_counts()

In [ ]:
def create_mlp(dim, regularizer=None):
  
    model = Sequential()
    model.add(Dense(13, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    return model

def create_cnn(width, height, filters=(64,32), regularizer=None):

    inputShape = (height, width)
    inputs = Input(shape=inputShape)
 
    for (i, f) in enumerate(filters):

        if i == 0:
            x = inputs

        x = Conv1D(f, (3), padding="same")(x)
        x = MaxPooling1D(pool_size=(2))(x)
    
    x = Flatten()(x)
    x = Dense(13,activation="relu", kernel_regularizer=regularizer)(x)
    model = Model(inputs, x)
    return model      

In [ ]:
strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X_train, Y_train)

for k, (train, test) in enumerate(kfold):

  X_tr=X_train[train,:]
  Y_tr=Y_train[train]
  X_te=X_train[test,:]
  Y_te=Y_train[test]
  
  
  labelencoder=LabelEncoder()
  y_train=to_categorical(labelencoder.fit_transform(Y_tr))
  labelencoder=LabelEncoder()
  y_test=to_categorical(labelencoder.fit_transform(Y_te))
  globalx_train=[]
  localx_train=[]
  globalx_test=[]
  localx_test=[] 

  for i in range(len(X_tr)):
    globalx_train.append(X_tr[i][1])
    localx_train.append(X_tr[i][0])

  for i in range(len(X_te)):
    globalx_test.append(X_te[i][1])
    localx_test.append(X_te[i][0])

  globalx_train=np.array(globalx_train)
  localx_train=np.array(localx_train)
  globalx_test=np.array(globalx_test)
  localx_test=np.array(localx_test)

  mlp = create_mlp(13)
  cnn = create_cnn(40,128)
  
  combinedInput = concatenate([mlp.output, cnn.output])

  x=Dense(20, activation="relu")(combinedInput)
  x = Dense(100, activation="relu")(x)
  x = Dense(4, activation="softmax")(x)

  model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)


  opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
  model1.compile(loss="categorical_crossentropy", metrics=['acc'], optimizer=opt)
  earlyStopping = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')
  mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
  reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4, mode='min')

  model1_history = model1.fit( 
  [globalx_train, localx_train], 
  np.array(y_train), 
  validation_data=([globalx_test, localx_test],np.array(y_test)), 
  epochs=50,callbacks=[mcp_save, reduce_lr_loss,earlyStopping])
  
  print('\n')
  print("fold",k)
  print("Accuracy of the model on Training Data is - " , model1.evaluate([globalx_train,localx_train], y_train)[1]*100 , "%")
  print("Accuracy of the model on Testing Data is - " , model1.evaluate([globalx_test, localx_test], y_test)[1]*100 , "%")
  print('\n')

  y_predict=(model1.predict([globalx_test, localx_test]) > 0.5).astype("int32")
  y_true=y_test
  cm= skm.multilabel_confusion_matrix(y_true, y_predict)
  print(skm.classification_report(y_true,y_predict))
  print('\n')

In [ ]:
dict(zip(labelencoder.classes_,range(len(labelencoder.classes_))))